In [1]:
!source ~/.bash_profile
%env PGDATABASE=postgres
%env PGHOST=10.128.0.5
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD=JordanGrahamRepo
%env PGINSTANCE=pg

env: PGDATABASE=postgres
env: PGHOST=10.128.0.5
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=JordanGrahamRepo
env: PGINSTANCE=pg


In [2]:
%env BASE_DIR=/home/jupyter/JordanGrahamRepo

env: BASE_DIR=/home/jupyter/JordanGrahamRepo


In [15]:
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:2: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [9]:
pip install psycopg2

  Using cached psycopg2-2.9.3.tar.gz (380 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      running egg_info
      creating /tmp/pip-pip-egg-info-3wctr6t5/psycopg2.egg-info
      writing /tmp/pip-pip-egg-info-3wctr6t5/psycopg2.egg-info/PKG-INFO
      writing dependency_links to /tmp/pip-pip-egg-info-3wctr6t5/psycopg2.egg-info/dependency_links.txt
      writing top-level names to /tmp/pip-pip-egg-info-3wctr6t5/psycopg2.egg-info/top_level.txt
      writing manifest file '/tmp/pip-pip-egg-info-3wctr6t5/psycopg2.egg-info/SOURCES.txt'
      
      Error: pg_config executable not found.
      
      pg_config is required to build psycopg2 from source.  Please add the directory
      containing pg_config to the $PATH or specify the full executable path with the
      option:
      
          python setup.py build_ext --pg-config /path/to/pg_config build

In [10]:
#Couldn't install psycopg2 so I installed this instead
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [16]:
%%timeit
import psycopg2 as psycopg, psycopg2.extras as p
import csv

host = '10.128.0.2'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        rows = []
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                rows.append(row)
                count += 1
                if len(rows) == 5000:
                    p.execute_batch(cur, sql, rows)
                    conn.commit()
                    print(count, "records inserted successfully into Person table")
                    rows = []

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    cur.close()
    conn.close()
    print("Postgres connection is closed")


<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
5000 records inserted successfully into Person table
10000 records inserted successfully into Person table
15000 records inserted successfully into Person table
20000 records inserted successfully into Person table
25000 records inserted successfully into Person table
30000 records inserted successfully into Person table
35000 records inserted successfully into Person table
40000 records inserted successfully into Person table
45000 records inserted successfully into Person table
50000 records inserted successfully into Person table
55000 records inserted successfully into Person table
60000 records inserted successfully into Person table
65000 records inserted successfully into Person table
70000 records inserted successfully into Person table
75000 records inserted successfully into Person table
80000 records inserted successfully into Person table
85000 recor

In [17]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)



In [18]:
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:2: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [20]:
%%timeit
#50000 batch size test
import psycopg2 as psycopg, psycopg2.extras as p
import csv

host = '10.128.0.2'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        rows = []
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                rows.append(row)
                count += 1
                if len(rows) == 50000:
                    p.execute_batch(cur, sql, rows)
                    conn.commit()
                    print(count, "records inserted successfully into Person table")
                    rows = []

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    cur.close()
    conn.close()
    print("Postgres connection is closed")

<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
50000 records inserted successfully into Person table
100000 records inserted successfully into Person table
150000 records inserted successfully into Person table
200000 records inserted successfully into Person table
250000 records inserted successfully into Person table
300000 records inserted successfully into Person table
350000 records inserted successfully into Person table
400000 records inserted successfully into Person table
450000 records inserted successfully into Person table
500000 records inserted successfully into Person table
550000 records inserted successfully into Person table
600000 records inserted successfully into Person table
650000 records inserted successfully into Person table
700000 records inserted successfully into Person table
750000 records inserted successfully into Person table
800000 records inserted successfully into Person t

In [21]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)



In [22]:
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:2: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [23]:
%%timeit
#500000 batch size test
import psycopg2 as psycopg, psycopg2.extras as p
import csv

host = '10.128.0.2'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        rows = []
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                rows.append(row)
                count += 1
                if len(rows) == 500000:
                    p.execute_batch(cur, sql, rows)
                    conn.commit()
                    print(count, "records inserted successfully into Person table")
                    rows = []

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    cur.close()
    conn.close()
    print("Postgres connection is closed")

<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records in

In [24]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)



In [25]:
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:2: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [29]:
%%timeit
#500000 C2 batch size test
import psycopg2 as psycopg, psycopg2.extras as p
import csv

host = '10.128.0.2'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        rows = []
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                rows.append(row)
                count += 1
                if len(rows) == 500000:
                    p.execute_batch(cur, sql, rows)
                    conn.commit()
                    print(count, "records inserted successfully into Person table")
                    rows = []

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    cur.close()
    conn.close()
    print("Postgres connection is closed")

<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7f6ff47e7c20; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.2 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records in

In [2]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)



In [4]:
#Count
!psql -f $BASE_DIR/create_table.sql

psql:/home/jupyter/JordanGrahamRepo/create_table.sql:2: NOTICE:  drop cascades to table load_testing.person
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE


In [1]:
%%timeit
#500000 High Bandwidth batch size test
import psycopg2 as psycopg, psycopg2.extras as p
import csv

host = '10.128.0.5'
password = 'JordanGrahamRepo'
url = "dbname='postgres' user='postgres' host={} port=5432 password={}".format(host, password)

conn = psycopg.connect(url)

print(conn)

sql = "INSERT INTO load_testing.Person (first_name, last_name, company_name, address, city, county, state, zip, phone1,\
       phone2, email, web) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

try:
    with open('us-1000000.csv', newline='') as csvfile:
        csv_reader = csv.reader(csvfile, delimiter=',')
        columns = next(csv_reader)
        rows = []
        count = 0
        with conn.cursor() as cur:
            for row in csv_reader:
                rows.append(row)
                count += 1
                if len(rows) == 500000:
                    p.execute_batch(cur, sql, rows)
                    conn.commit()
                    print(count, "records inserted successfully into Person table")
                    rows = []

except psycopg.errors.OperationalError as error:
    print("Failed to insert record into Person table {}".format(error))
                
finally:
    cur.close()
    conn.close()
    print("Postgres connection is closed")

<connection object at 0x7fea7d5c3590; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.5 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7fea7d5c3590; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.5 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7fea7d5c3590; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.5 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records inserted successfully into Person table
Postgres connection is closed
<connection object at 0x7fea7d5c3590; dsn: 'user=postgres password=xxx dbname=postgres host=10.128.0.5 port=5432', closed: 0>
500000 records inserted successfully into Person table
1000000 records in

In [3]:
#Count
!psql -c "select count(*) from load_testing.Person"

  count  
---------
 8000000
(1 row)

